<a href="https://colab.research.google.com/github/KARNAMSAI/KARNAMSAI/blob/main/recon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
import numpy as np
import pandas as pd

In [45]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.head()

In [ ]:
credits.head()


In [46]:
movies=movies.merge(credits,on='title')

In [48]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.shape

In [ ]:
movies.dropna(inplace=True)
movies.duplicated().sum()

In [73]:
def convert(obj):
  l=[]
  for i in ast.literal_eval(obj):
    l.append(i['name'])
  return l

In [68]:
def convert3(obj):
     l=[]
     counter=0
     for i in ast.literal_eval(obj):
        if counter !=3:
          l.append(i['name'])
        else:
          break
     return l

In [74]:
def fetch_director(obj):
  l=[]
  for i in ast.literal_eval(obj):
    if i['job']=='Director':
      l.append(i['name'])
      break
  return l

In [63]:
import ast

In [66]:
movies['genres']=movies['genres'].apply(convert)

In [67]:
movies['keywords']=movies['keywords'].apply(convert)

In [71]:
movies['cast']=movies['cast'].apply(convert3)

In [ ]:
movies.head()

In [76]:
movies['crew']=movies['crew'].apply(fetch_director)

In [78]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [81]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","")for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","")for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","")for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","")for i in x])

In [83]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [85]:
new_df=movies[['movie_id','title','tags']]

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df.head()

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

In [98]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [102]:
def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))

  return " ".join(y)

In [ ]:
new_df['tags']=new_df['tags'].apply(stem)

In [106]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [113]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [112]:
from sklearn.metrics.pairwise import cosine_similarity
similar=cosine_similarity(vectors)

In [ ]:
sorted(list(enumerate(similar[0])),reverse=True,key=lambda x:x[1])[1:6]

In [134]:
def recommend(movie):
  movie_index=new_df[new_df['title']==movie].index[0]
  distances=similar[movie_index]
  movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
  for i in movies_list:
    print(new_df.iloc[i[0]].title)

In [136]:
recommend('Batman Begins')

The Dark Knight
Batman
The Dark Knight Rises
Batman
Rockaway
